# Stable Diffusion in KerasCV



## Overview

KerasCV 라이브러리에 구현된 [stability.ai](https://stability.ai/) 의 text-to-image 모델,
[Stable Diffusion](https://github.com/CompVis/stable-diffusion) 을 사용해 봅니다.

Stable Diffusion 은 이미지 생성 분야의 지각변동을 일으킨 **오픈소스** text-to-image 생성모델입니다.  
오픈소스 프로젝트이기 때문에, 여러 프로젝트에 구현체들이 존재합니다.
KerasCV 의 구현체는 [XLA compilation](https://www.tensorflow.org/xla) 과
[mixed precision](https://www.tensorflow.org/guide/mixed_precision) 기능을 이용하여 속도를 올렸습니다.  
해당 기능들은 Tensorflow 플랫폼에서 지원합니다.  

직접 Stable Diffusion 모델을 받아서 사용해보도록 하겠습니다.  

In [ ]:
# windows 에서 module import error 발생. resource module 을 windows 에서 호환되도록 준비해야함. colab 에서 할 것.

!pip install tensorflow keras_cv --upgrade --quiet

In [ ]:
import time
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt

## Introduction

`keras_cv.models.StableDiffusion()` method 를 이용하여 미리 준비된 Stable Diffusion 모델을 가져옵니다.


In [ ]:
model = keras_cv.models.StableDiffusion(img_width=512, img_height=512)

프롬프트에 아래와 같은 문구를 넣어봅시다.

In [ ]:
images = model.text_to_image("photograph of an astronaut (Samsung Electronics) riding a horse", batch_size=3)


def plot_images(images):
    plt.figure(figsize=(20, 20))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i])
        plt.axis("off")


plot_images(images)

보다 복잡한 prompt 도 넣어봅시다.  
Keyword 들을 잘 나열하면 원하는 결과물을 얻기 쉽습니다.  

In [ ]:
images = model.text_to_image(
    "cute magical flying dog, fantasy art, "
    "golden color, high quality, highly detailed, elegant, sharp focus, "
    "concept art, character concepts, digital painting, mystery, adventure",
    batch_size=3,
)
plot_images(images)

## 동작은 어떻게 되는 것인가요?



Stalbe Diffusion 은 아래 3가지 파트로 요약할 수 있습니다.

- Text Encoder, 프롬프트를 인코딩하여 vector 로 만듧니다.
- Diffusion Model, 디노이징을 해서 64x64 이미지 vector (Latent Image Patch) 를 만듦니다.
- Decoder, 64x64 Image Patch를 고화질 512x512 이미지로 만듦니다.

디노이징은 step을 반복하는데, step을 많이 할 수록 이미지 품질이 좋아집니다.  
기본 셋팅은 50 스텝으로 되어있습니다.


![The Stable Diffusion architecture](https://i.imgur.com/2uC8rYJ.png)

실제 keras 의 구현은 아래 코드에서 볼 수 있고, 매우 간단합니다.


- [text_encoder.py](https://github.com/keras-team/keras-cv/blob/master/keras_cv/models/stable_diffusion/text_encoder.py): 87 LOC
- [diffusion_model.py](https://github.com/keras-team/keras-cv/blob/master/keras_cv/models/stable_diffusion/diffusion_model.py): 181 LOC
- [decoder.py](https://github.com/keras-team/keras-cv/blob/master/keras_cv/models/stable_diffusion/decoder.py): 86 LOC
- [stable_diffusion.py](https://github.com/keras-team/keras-cv/blob/master/keras_cv/models/stable_diffusion/stable_diffusion.py): 106 LOC



## KerasCV 구현체 장점 - 가속하기

KerasCV 말고도 Stable Diffusion 의 구현체는 여러가지가 있습니다.  
KerasCV 의 장점은 아래와 같습니다.

- Graph mode execution
- XLA compilation through `jit_compile=True`
- Support for mixed precision computation

[HuggingFace diffusers](https://github.com/huggingface/diffusers) 구현체와 시간을 비교해보겠습니다.

Tesla T4 GPU 에서 수행했습니다.


| GPU        | Model                  | Runtime   |
|------------|------------------------|-----------|
| Tesla T4   | KerasCV (Warm Start)   | **28.97s**|
| Tesla T4   | diffusers (Warm Start) | 41.33s    |
| Tesla V100 | KerasCV (Warm Start)   | **12.45** |
| Tesla V100 | diffusers (Warm Start) | 12.72     |

| GPU        | Model                  | Runtime |
|------------|------------------------|---------|
| Tesla T4   | KerasCV (Cold Start)   | 83.47s  |
| Tesla T4   | diffusers (Cold Start) | 46.27s  |
| Tesla V100 | KerasCV (Cold Start)   | 76.43   |
| Tesla V100 | diffusers (Cold Start) | 13.90   |


**참고, 하드웨어 셋업에 따라 많이 다를 수 있습니다.**

### Mixed precision 사용하기

"Mixed precision" 은 실제 연산시에 `float16` 타입을 사용합니다.
`float32` 를 사용할 때 보다 당연히 연산이 효율적입니다. (요즘의 NVIDIA GPUs 기준)

In [ ]:
keras.mixed_precision.set_global_policy("mixed_float16")

위 코드로 설정만 바꿔주면 끝입니다. 확인해봅시다.

In [ ]:
model = keras_cv.models.StableDiffusion()

print("Compute dtype:", model.diffusion_model.compute_dtype)
print(
    "Variable dtype:",
    model.diffusion_model.variable_dtype,
)

images = model.text_to_image("An avocado armchair", batch_size=3)
plot_images(images)

### XLA Compilation 사용하기

TensorFlow 는 [XLA: Accelerated Linear Algebra](https://www.tensorflow.org/xla) 컴파일러를 지원합니다.  
`keras_cv.models.StableDiffusion` 은 `jit_compile` 옵션을 지원합니다.  
이 옵션을 사용하면 `True`  XLA compilation 기능을 사용하게 되고, 눈에 띄게 속도가 빨라집니다.


In [ ]:
model = keras_cv.models.StableDiffusion(jit_compile=True)
# Before we benchmark the model, we run inference once to make sure the TensorFlow
# graph has already been traced.
images = model.text_to_image("An avocado armchair", batch_size=3)
plot_images(images)

A100 GPU 기준 2배정도 빨라진다고 합니다.

## 참고


NVIDIA GPU 또는 AppleSilicon Mac 이 있다면, local 에서 돌려볼 수도 있습니다.  
(2022년 기준, M1 MacBookPro 는 mixed precision 지원하지 않네요... ㅠㅠ)

(원 저자)  
**Authors:** [fchollet](https://twitter.com/fchollet), [lukewood](https://twitter.com/luke_wood_ml), [divamgupta](https://github.com/divamgupta)<br>
**Date created:** 2022/09/25<br>
**Last modified:** 2022/09/25<br>